<a href="https://colab.research.google.com/github/agu3/ds4bmeTest/blob/master/a8_q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling

In [0]:
shhs = pd.read_csv('shhs1.txt', sep = '\t')
shhs1 = shhs.dropna()

shhs1['rdi4pNEW'] = [1 if x > 7 else 0 for x in shhs1['rdi4p']]
shhs1 = shhs1.drop(columns = 'rdi4p')
shhs1 = shhs1.drop(columns = 'pptid')
shhs1.head()

In [0]:
train_dataset = shhs1.sample(frac = 0.8, random_state = 0)
test_dataset = shhs1.drop(train_dataset.index)

In [0]:
train_stats = train_dataset.describe()
train_stats.pop('rdi4pNEW')
train_stats = train_stats.transpose()
train_stats

In [0]:
train_labels = train_dataset.pop('rdi4pNEW')
test_labels = test_dataset.pop('rdi4pNEW')

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [0]:
normed_test_data = normed_test_data.drop(columns = 'Staging5')
normed_train_data = normed_train_data.drop(columns = 'Staging5')

normed_test_data.isna().sum()
normed_train_data.isna().sum()

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation = 'relu', input_shape = (normed_train_data.shape[0], normed_test_data.shape[1])),
  tf.keras.layers.Dense(64, activation = 'relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation = 'sigmoid'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])

model.compile(
    loss = 'mse',
    optimizer = 'adam',
    metrics = ['accuracy']
)
    

In [309]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 2318, 64)          1792      
_________________________________________________________________
dense_58 (Dense)             (None, 2318, 64)          4160      
_________________________________________________________________
dropout_8 (Dropout)          (None, 2318, 64)          0         
_________________________________________________________________
dense_59 (Dense)             (None, 2318, 32)          2080      
_________________________________________________________________
dropout_9 (Dropout)          (None, 2318, 32)          0         
_________________________________________________________________
dense_60 (Dense)             (None, 2318, 1)           33        
Total params: 8,065
Trainable params: 8,065
Non-trainable params: 0
___________________________________________________

In [310]:
history = model.fit(
    normed_train_data, train_labels, epochs = 1000, validation_split = 0.2,
    verbose = 0,
    callbacks=[tfdocs.modeling.EpochDots()]
)


Epoch: 0, accuracy:0.5431,  loss:0.3692,  val_accuracy:0.6961,  val_loss:0.1962,  
....................................................................................................
Epoch: 100, accuracy:0.9288,  loss:0.0702,  val_accuracy:0.6509,  val_loss:0.2549,  
....................................................................................................
Epoch: 200, accuracy:0.9693,  loss:0.0372,  val_accuracy:0.6659,  val_loss:0.2703,  
....................................................................................................
Epoch: 300, accuracy:0.9871,  loss:0.0194,  val_accuracy:0.6573,  val_loss:0.2766,  
....................................................................................................
Epoch: 400, accuracy:0.9908,  loss:0.0160,  val_accuracy:0.6487,  val_loss:0.2860,  
....................................................................................................
Epoch: 500, accuracy:0.9908,  loss:0.0136,  val_accuracy:0.6530,  val_l

In [312]:
model.evaluate(normed_test_data, test_labels, verbose=2)

19/19 - 0s - loss: 0.3162 - accuracy: 0.6356


[0.31620919704437256, 0.6355785727500916]

My test set accuracy using this neural network was  0.6356

In [0]:
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.datasets import make_regression
from sklearn.linear_model import ElasticNet

In [0]:
shhs2 = pd.read_csv('shhs1.txt', sep = '\t')
shhs2 = shhs.dropna()

shhs2['rdi4pNEW'] = [1 if x > 7 else 0 for x in shhs2['rdi4p']]
shhs2 = shhs2.drop(columns = 'rdi4p')
shhs2 = shhs2.drop(columns = 'pptid')
shhs2.head()

In [0]:
y = shhs2.pop('rdi4pNEW')
x = shhs2

In [0]:
x, y = make_regression(n_features = 29, random_state = 0)
lg = ElasticNet(random_state = 0)
lg.fit(x, y)
print(lg.coef_)

In [0]:
y_predict = lg.predict(x)

In [341]:
lg.score(x, y)

0.8798182406895314

My test set accuracy using regression is 0.8798182406895314. For some reason, this is higher than my test set accuracy using the neural network. I think it's highly likely that I overfit my model to the training data, but I'm not sure where that might have happened. 